In [40]:
import pandas as pd
import zipfile
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import torch


In [30]:
z = zipfile.ZipFile('data/Sentiment140 dataset with 1.6 million tweets.zip')
f = z.open(z.filelist[0])
df = pd.read_csv(f,
                 encoding='latin-1',
                 header=None,
                 names=['label', 'id', 'date', 'query', 'user', 'text'])
df.label = df.label.apply(lambda x: int(x / 4))
f.close()
z.close()


In [31]:
# vader = SentimentIntensityAnalyzer()
df = df[['text', 'label']]


In [32]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.label.values,
                                                  test_size=0.15,
                                                  random_state=42,
                                                  stratify=df.label.values)


In [33]:
df['data_type'] = df.shape[0] * ['not_set']
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'


In [34]:
df.groupby(['label','data_type']).count()

text
label data_type        
0     train      680000
      val        120000
1     train      680000
      val        120000

In [37]:
tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path='bert-base-uncased', do_lower_case=True)


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [38]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type == 'train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt')

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type == 'val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt')


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/h/heziyang/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/h/heziyang/.local/lib/python3.8/site-packages/transformers/tokenization_utils_bas

In [42]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type == 'val'].label.values)
